In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 76kB/s 
     |████████████████████████████████| 204kB 53.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=1a2a9a0241ccc9bff306c137b7782af1ac9fc9c26fce074546d408e447f7e464
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import numpy as np
sc = SparkContext()
#spark = SparkSession(sc)


In [5]:
def pair(x):
  return x,-1

In [19]:
file1 = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Assignment-6/email-Eu-core.txt'
fileRDD = sc.textFile(file1)
fileRDD1=fileRDD.flatMap(lambda line: line.split(" "))
fileRDD2=fileRDD1.map(lambda line: int(line))
fileRDD3=fileRDD2.distinct()
fileRDD3=fileRDD3.map(lambda x: pair(x))
n=fileRDD2.max()+1

print(fileRDD3.take(1))

[(0, -1)]


In [20]:
def stringtoint(line):
  return int(line[0]),int(line[1])

In [21]:
def stringtoint2(line):
  return int(line[1]),int(line[0])

In [22]:
def creatematrix(pair):
  li=np.zeros(n)
  for x in list(pair[1]):
    if x>-1:
      li[x]=1
  return pair[0],li 

In [23]:

RDD1=fileRDD.map(lambda line: line.split(" "))
RDD2=RDD1.map(lambda line: stringtoint(line))
#RDD2=RDD2.groupByKey()
RDDJ=fileRDD3.union(RDD2)
RDD3=RDDJ.groupByKey()
LinkM=RDD3.map(lambda x: creatematrix(x))

L=sc.broadcast(LinkM.collect())


In [24]:
RDD4=RDD1.map(lambda line: stringtoint2(line))
RDDJ2=RDD4.union(fileRDD3)
RDD5=RDDJ2.groupByKey()
LinkMT=RDD5.map(lambda x: creatematrix(x))
LT=sc.broadcast(LinkMT.collect())

In [25]:
len(LT.value)

1005

In [44]:
A = np.ones(n, dtype=np.float64).reshape(n, 1)
H = np.ones(n , dtype=np.float64).reshape(n, 1)
for i in range(10):
  print(i)
  for k in range(len(LT.value)):
    Link=LT.value[k]
    arr=np.array(Link[1]).reshape(1,n)
    A[int(Link[0])]=arr.dot((H))

  for k in range(len(L.value)):
    Lin=L.value[k]
    arr=np.array(Lin[1]).reshape(1,n)
    H[Lin[0]]=arr.dot((A))
  
  # H = H/np.max(H)
  # A = A/np.max(A)

0
1
2
3
4
5
6
7
8
9


In [45]:
from operator import itemgetter

tuple_hubs = []
tuple_auths = []

for i in range(0, 1005):
  t = (i, H[i,0])
  tuple_hubs.append(t)

  t = (i, A[i,0])
  tuple_auths.append(t)

sorted_hubs_asc = sorted(tuple_hubs,key=itemgetter(1), reverse=False)
sorted_hubs_dsc = sorted(tuple_hubs,key=itemgetter(1), reverse=True)

sorted_auths_asc = sorted(tuple_auths,key=itemgetter(1), reverse=False)
sorted_auths_dsc = sorted(tuple_auths,key=itemgetter(1), reverse=True)

In [46]:
print('Top nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_dsc[i][0], np.round(sorted_hubs_dsc[i][1],4)))

Top nodes IDs with hub scores
1     160   6.069371737960707e+36 
2     82    5.491410684772275e+36 
3     121   5.442121029433448e+36 
4     107   5.018255256872204e+36 
5     62    4.7010653098295124e+36 


In [41]:
print('Bottom nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_asc[i][0], np.round(sorted_hubs_asc[i][1],4)))

Bottom nodes IDs with hub scores
1     78    0.0   
2     203   0.0   
3     239   0.0   
4     289   0.0   
5     315   0.0   


In [42]:
print('Top nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_dsc[i][0], np.round(sorted_auths_dsc[i][1],4)))

Top nodes IDs with authority scores
1     160   1.0   
2     107   0.9554 
3     62    0.9273 
4     434   0.8982 
5     121   0.8963 


In [43]:
print('Bottom nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_asc[i][0], np.round(sorted_auths_asc[i][1],4)))

Bottom nodes IDs with authority scores
1     524   0.0   
2     750   0.0   
3     755   0.0   
4     790   0.0   
5     858   0.0   


In [27]:
H[1]

array([0.00272887])

In [28]:
np.argmax(A)

160

In [29]:
np.argmin(A)

524

In [30]:
np.argmax(H)

160

In [ ]:
np.argmin(H)

78